In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nvidia/NV-Embed-v2", trust_remote_code=True)
model.max_seq_length = 32768
model.tokenizer.padding_side = "right"

In [ ]:
def get_embeddings(texts):
    padded_texts = [i + model.tokenizer.eos_token for i in texts]

    embeddings = model.encode(
        padded_texts,
        normalize_embeddings=True,
        batch_size=1,
    )
    
    return embeddings.tolist()


In [ ]:
import faiss
import numpy as np

def get_similarities(query_embedding, embeddings):

    query_embedding = np.array(query_embedding, dtype=np.float32).reshape(1, -1)
    embeddings = np.array(embeddings, dtype=np.float32)

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings)  # type: ignore

    scores, indices = index.search(query_embedding, len(embeddings))  # type: ignore

    return scores, indices


In [ ]:
query_text = "arguments with Estela, emotional tension, unresolved conflicts, recent disagreements"
texts = [
    "Estela is a curious person who seeks to understand the design of her work environment.",
    "Giovanni and Estela are resolving a recent conflict",
    "Giovanni and Estela had a disagreement the previous day"
]

query_embedding = get_embeddings([query_text])[0]
embeddings = get_embeddings(texts)

scores, indices = get_similarities(query_embedding, embeddings)

print(scores)
print(indices)